Conversion de PDF a JPEG

In [41]:
# import module
from pdf2image import convert_from_path
import cv2
import numpy as np


# Store Pdf with convert_from_path function
images = convert_from_path('cc2.pdf',500,poppler_path=r'C:\Program Files\poppler-22.11.0\Library\bin')

for i in range(len(images)):
	# Save pages as images in the pdf
	images[i].save('page'+ str(i) +'.jpg', 'JPEG')

PROCESAMIENTO DE IMÁGENES

In [43]:
# LECTURA, REDIMENSIONAMIENTO Y FILTRADO DE IMAGENES
# DOCUMENTO
kernel = np.ones((5,5), np.uint8)
img = cv2.imread('page0.JPG')
img = cv2.resize(img,(800,750))
img_gr = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img_can = cv2.Canny(img,200,200)
# PLANTILLAS
# FRONTAL
pl_frontal = cv2.imread('plantilla_frontal.JPG')
pl_frontal = cv2.resize(pl_frontal,(500,300))
pl_frontal_gr = cv2.cvtColor(pl_frontal,cv2.COLOR_BGR2GRAY)
pl_frontal_can = cv2.Canny(pl_frontal,200,200)
# TRASERA
pl_trasera = cv2.imread('plantilla_trasera.JPG')
pl_trasera = cv2.resize(pl_trasera,(600,300))
pl_trasera_gr = cv2.cvtColor(pl_trasera,cv2.COLOR_BGR2GRAY)
pl_trasera_can = cv2.Canny(pl_trasera,200,200)

# MASCARA
# cv2.imshow('CANNY', img_can)
img_dil = cv2.dilate(img_can,kernel,iterations=1)
# cv2.imshow('DILATADO', img_dil)
img_eroded = cv2.erode(img_dil,kernel,iterations=1)
# cv2.imshow('ERODED', img_eroded)
mascara = cv2.adaptiveThreshold(img_dil, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,55,24)
# mascara = cv2.adaptiveThreshold(img_can, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,13,35)
# cv2.imshow('MASCARA', mascara)


# IDENTIFICAR CANTIDAD DE CARAS DEL DOCUMENTO EN LA PÁGINA
contornos, _ = cv2.findContours(mascara,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
s=1
for cont in contornos:
    rectangulo = cv2.minAreaRect(cont)
    (x,y), (an, al), angulo = rectangulo
    if(al*an > 50000 ):
        # print('area: ',al*an)
        rect = cv2.boxPoints(rectangulo)
        rect = np.int0(rect)
        cv2.polylines(img,[rect], True,(0,255,0), 2)
        print(rect)
        x_min = rect[0][0]
        x_max = rect[0][0]
        y_min = rect[0][1]
        y_max = rect[0][1]
        for pun in rect:
            if(pun[0] < x_min):
                x_min = pun[0]
            elif(pun[0] > x_max):
                x_max = pun[0]
            if(pun[1] < y_min):
                y_min = pun[1]
            elif(pun[1] > y_max):
                y_max = pun[1]
        print('Xmin',x_min,'Xmax',x_max)
        print('Ymin',y_min,'Ymax',y_max)
        img_cort = img[y_min:y_max,x_min:x_max]
        nm='CORTADA'+str(s)
        cv2.imshow(nm,img_cort)
        s+=1

mascara2 = cv2.adaptiveThreshold(pl_frontal_gr, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,13,12)
# cv2.imshow('MASCARA2', mascara2)


# COMPARACIÓN DE IMAGENES
# resultado = cv2.matchTemplate(img_can,pl_trasera_can,cv2.TM_CCOEFF_NORMED)
resultado = cv2.matchTemplate(mascara,mascara2,cv2.TM_CCOEFF_NORMED)
# print(resultado)
min, max, pos_min, pos_max =cv2.minMaxLoc(resultado)

pix_sup_izq = pos_max
pix_inf_der = (pos_max[0]+500, pos_min[1]+50)
print(max,min )

# cv2.rectangle(img, pix_sup_izq, pix_inf_der,255 ,4 )
cv2.imshow('RESULTADO',img)
# cv2.imshow('PLANTILLA',pl_trasera_can)

# cv2.imshow('ORIGINAL', img)
# cv2.imshow('CANNY', img_can)
# cv2.imshow('GRISES',img_gr)
# cv2.imshow('ORIGINAL F', pl_frontal)
# cv2.imshow('CANNY F', pl_frontal_can)
# cv2.imshow('GRISES F',pl_frontal_gr)
# cv2.imshow('ORIGINAL T', pl_trasera)
# cv2.imshow('CANNY T', pl_trasera_can)
# cv2.imshow('GRISES T',pl_trasera_gr)
cv2.waitKey(0)


[[ 14 728]
 [ 14 392]
 [672 392]
 [672 728]]
Xmin 14 Xmax 672
Ymin 392 Ymax 728
[[ 50  86]
 [659  82]
 [661 346]
 [ 52 349]]
Xmin 50 Xmax 661
Ymin 82 Ymax 349
0.09560928493738174 -0.11489830911159515


-1